In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import json
import numpy as np
import pandas as pd
from params import *

from tqdm import tqdm
from itertools import combinations
from utils.read2df import read2df
from utils.cointncorr import CointnCorr

In [2]:
# %%capture
if symbols is None:
    !python binance-public-data/python/download-kline.py \
        -i {" ".join(list(freqs.keys()))} -startDate {start_date} -t spot -skip-daily 1
else:
    !python binance-public-data/python/download-kline.py \
        -s {" ".join(symbols)} -i {" ".join(list(freqs.keys()))} -startDate {start_date} -t spot -skip-daily 1

Found 3 symbols
[1/3] - start download monthly BTCUSD klines 

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSD/1m/BTCUSD-1m-2022-01.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSD/1m/BTCUSD-1m-2022-02.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSD/1m/BTCUSD-1m-2022-03.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSD/1m/BTCUSD-1m-2022-04.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSD/1m/BTCUSD-1m-2022-05.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSD/1m/BTCUSD-1m-2022-06.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSD/1m/BTCUSD-1m-2022-07.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSD/1m/BTCUSD-1m-2022-08.zip

File not found: https://data.binance.vision/data/spot/monthly/klines/BTCUSD/1m/BTCUSD-1m-2022-09.zip

File not found: htt

In [3]:
'''
Read the downloaded OHLCV data into `pandas` dataframe
'''

# dfs = read2df(symbols, freqs)
dfs = read2df(symbols, freqs)

# have a preview
dfs[0]

,time,open,high,low,close,volume,tic,itvl,datetime
0,1640995259999,40634.64,40675.66,40634.36,40671.11,1.76233,BTCEUR,1m,2022-01-01 00:00:59.999
1,1640995259999,34240.67,34286.76,34240.67,34264.35,0.04050,BTCGBP,1m,2022-01-01 00:00:59.999
2,1640995319999,40671.00,40730.46,40664.00,40724.23,1.48283,BTCEUR,1m,2022-01-01 00:01:59.999
3,1640995319999,34283.72,34302.14,34279.06,34283.09,0.02792,BTCGBP,1m,2022-01-01 00:01:59.999
4,1640995379999,40732.08,40773.15,40691.92,40762.47,2.89927,BTCEUR,1m,2022-01-01 00:02:59.999
...,...,...,...,...,...,...,...,...,...
1926553,1698796679999,27977.60,27977.60,27977.60,27977.60,0.00000,BTCGBP,1m,2023-10-31 23:57:59.999
1926554,1698796739999,32794.22,32805.03,32794.22,32800.03,0.00843,BTCEUR,1m,2023-10-31 23:58:59.999
1926555,1698796739999,27977.60,27977.60,27977.60,27977.60,0.00000,BTCGBP,1m,2023-10-31 23:58:59.999
1926556,1698796799999,32805.73,32805.73,32787.34,32790.79,0.08751,BTCEUR,1m,2023-10-31 23:59:59.999


In [6]:
'''
Set data before `trade_data` as training data, after `trade_data` is trade_data
'''

trains, tests = [], []
for i in range(len(dfs)):
    trains.append(dfs[i][(dfs[i]['datetime'] > start_date) & (dfs[i]['datetime'] < trade_date)].reset_index(drop=True))
    tests.append(dfs[i][dfs[i]['datetime'] >= trade_date].reset_index(drop=True))

In [7]:
'''
Find the best pair
'''

import pickle

tables = CointnCorr(trains, freqs).tabulate()
cointncorrtxt = f"result/gridsearch/cointncorr.txt"

os.remove(f"{cointncorrtxt}") if os.path.exists(f"{cointncorrtxt}") else None

with open(cointncorrtxt, "w+") as f:
    for k, v in tables.items():
        f.write(f"{k}\n")
        f.write(f"{v}\n\n")
f.close()

best_value = 0
for key in tables.keys():
    for freq in freqs:
        rel = tables[key].at['coint', freq] + tables[key].at['corr', freq]
        if rel > best_value:
            best_value = rel
            best_pair = key
            best_freq = freq

print("===========================================")
print(f"Best trading pairs shall be: {best_pair} under {best_freq} interval")
print(f"the coint is {round(tables[best_pair].at['coint', best_freq]*100, 2)}%")
print(f"and the corr is {round(tables[best_pair].at['corr', best_freq],3)}")
print("===========================================")

cointncorr = best_pair.split("_"), best_freq

with open('result/cointncorr.pickle', 'wb') as pk:
    pickle.dump(cointncorr, pk)

Best trading pairs shall be: BTCEUR_BTCGBP under 1m interval
the coint is 34.37%
and the corr is 0.952
